##  Imports and Setup

In [1]:
import sys
import os
import json
from pathlib import Path

sys.path.append(str(Path().absolute().parent))  # Adjust as needed

from src.data_model import DataModel  # Updated module

# Load config
with open("../keys.json") as f:
    config = json.load(f)

sqlite_path = os.path.join(Path().absolute().parent, "data", "my_data.db")
print(sqlite_path)
vector_cfg = config["qdrant"]

# Initialize DataModel
data_model = DataModel(sqlite_path=sqlite_path, vector_cfg=vector_cfg)


/Users/saivarunreddymullangi/Desktop/agent/data/my_data.db


## Fetch All Tables and Collections

In [4]:
import sqlite3
from pathlib import Path
import os

db_path = os.path.join(Path().absolute().parent, "data", "my_data.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print("🧱 Raw Tables:", tables)

conn.close()


🧱 Raw Tables: [('users',), ('products',)]


In [5]:
import os
print("📁 Current notebook working directory:")
print(os.getcwd())


📁 Current notebook working directory:
/Users/saivarunreddymullangi/Desktop/agent/example_usage


In [2]:
# Get all SQLite tables
all_tables = data_model.fetch_tables()
print("📋 All Structured Tables:", all_tables)

# Get all Qdrant collections
all_collections = data_model.fetch_unstructured_collections()
print("📚 All Unstructured Collections:", all_collections)


[<sqlite3.Row object at 0x10676a560>, <sqlite3.Row object at 0x10676a2f0>]
[]
📋 All Structured Tables: []
📚 All Unstructured Collections: ['demo_qdrant', 'reviews', 'support_docs']


In [12]:
import requests

url = f"{structured_cfg['SUPABASE_URL']}/rest/v1/rpc/execute_sql?columns=name:text"
headers = {
    "apikey": structured_cfg["SUPABASE_API_KEY"],
    "Authorization": f"Bearer {structured_cfg['SUPABASE_API_KEY']}",
    "Content-Type": "application/json"
}

sql = """SELECT * FROM execute_sql('SELECT table_name FROM information_schema.tables
    WHERE table_schema = ''public''
    AND table_name NOT LIKE ''_%''') AS t(table_name text);"""

response = requests.post(url, headers=headers, json={"query": sql})

print("📡 Status Code:", response.status_code)
print("📄 Response Text:", response.text)


📡 Status Code: 404
📄 Response Text: {"code":"PGRST202","details":"Searched for the function public.execute_sql with parameter name or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.","hint":null,"message":"Could not find the function public.execute_sql(name) in the schema cache"}


In [9]:
url = f"{structured_cfg['SUPABASE_URL']}/rest/v1/rpc/execute_sql"
headers = {
    "apikey": structured_cfg["SUPABASE_API_KEY"],
    "Authorization": f"Bearer {structured_cfg['SUPABASE_API_KEY']}",
    "Content-Type": "application/json",
    "Prefer": "params=columns"  # ✅ Tells Supabase to use param-shaping
}

sql = "SELECT 'users' AS name"

# POST payload includes expected columns
payload = {
    "query": sql,
    "columns": [{"name": "name", "type": "text"}]
}

response = requests.post(url, headers=headers, json=payload)

print("📡 Status Code:", response.status_code)
print("📄 Response Text:", response.text)


📡 Status Code: 404
📄 Response Text: {"code":"PGRST202","details":"Searched for the function public.execute_sql with parameters columns, query or with a single unnamed json/jsonb parameter, but no matches were found in the schema cache.","hint":"Perhaps you meant to call the function public.execute_sql(query)","message":"Could not find the function public.execute_sql(columns, query) in the schema cache"}


In [4]:

# Initialize the DataModel with error handling
try:
    data_model = DataModel(structured_cfg, vector_cfg)
except Exception as e:
    print(f"Error initializing DataModel: {e}")
    raise

# Test database connections
try:
    all_tables = data_model.fetch_tables()
    print("📋 All Structured Tables:", all_tables)
except Exception as e:
    print(f"Error fetching tables: {e}")
    raise

try:
    all_collections = data_model.fetch_unstructured_collections()
    print("📚 All VectorDB Collections:", all_collections)
except Exception as e:
    print(f"Error fetching collections: {e}")
    raise

Error fetching tables: Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)